### Pytorch中实现正则化
`torch.optim`中集成了很多优化器，比如SGD、Adadelta、Adam、Adagrad、RMS皮绒皮等，这些优化器自带的参数weight_dacay，用于指定权值衰减率，相当于L2正则化中的$\lambda$参数

### Dropout正则化
在训练过程中按照一定比例随机忽略或者屏蔽一些神经元，也就是说它们在正向传播中对于下游神经元的作用暂时消失了，反向传播中它们的权重也不会更新，通过传播过程，Dropout将产生和L2范数相同的收缩权重的效果

神经元在训练中可能会对某些特征权重有特殊化，如果过度依赖这些特殊的权重，就会产生过拟合现象。这种过度依赖于上下文的现象被称为复杂的协同适应(complex co-adaptation)，Dropout就可以解决这个问题

使用dropout的原则
- dropout rate一般设置为0.2-0.5，比例太低起不到效果，比例太高欠拟合
- 在大的网络上应用
- 输入层和隐藏层都使用dropout
- 增加learning rate和momentum
- 限制网络模型的权重

下面是使用Dropout的例子

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
net_overfitting = torch.nn.Sequential(
    torch.nn.Linear(13,16),
    torch.nn.ReLU(),
    torch.nn.Linear(16,32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,1),
)

net_dropout = torch.nn.Sequential(
    torch.nn.Linear(13,16),
    torch.nn.Dropout(0.5),
    torch.nn.ReLU(),
    torch.nn.Linear(16,32),
    torch.nn.Dropout(0.5),
    torch.nn.ReLU(),
    torch.nn.Linear(32,1),
)

### Batch Normalization
为了解决梯度消失问题，因为sigmoid和tanh作为激活函数的时候，数据分布两侧的时候激活函数的导数接近于0

有两个学习参数$\gamma$和$\beta$，是用来优化特征分布的表达力，就是在标准化之后对其进行线性变化

BN是对隐藏层的标准化处理，和输入的标准化处理不同，输入的标准化处理是让所有的输入均值为0，方差为1，BN是让隐藏层输入的方差和均值为任意值。如果遇到收敛速度很慢，或者梯度爆炸无法训练等情况，可以尝试使用BN来解决

用在卷积和激活之间

In [4]:
net_BN = torch.nn.Sequential(
    torch.nn.Linear(13,16),
    torch.nn.BatchNorm1d(num_features=16),
    torch.nn.ReLU(),
    torch.nn.Linear(16,32),
    torch.nn.BatchNorm1d(num_features=32),
    torch.nn.ReLU(),
    torch.nn.Linear(32,1),
)

### 激活函数
比较深的网络一般选用relu，多分类用softmax

激活函数的输入和输出维度是一样的，输入维度一般包括批量数N，所以输入数据的维度一般是4维。比如(N,C,W,H)

### 损失函数
常用的有两种:Cross Entropy和Mean squared error，分别对应分类和回归问题


交叉熵反映了两个概率分布之间的距离

均方误差反映了预测值和实际值之间的误差，因为回归问题最后一般都是一个输出

In [8]:
torch.manual_seed(10)

loss = nn.MSELoss(reduction='mean') # 均值，如果是sum，不除以n
input = torch.randn(1,2,requires_grad=True)
print(input)
target = torch.randn(1,2)
print(target)
output = loss(input,target)
print(output)
output.backward()

tensor([[-0.6014, -1.0122]], requires_grad=True)
tensor([[-0.3023, -1.2277]])
tensor(0.0680, grad_fn=<MseLossBackward>)


In [11]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3,5,requires_grad=True)
print(input)
target = torch.empty(3,dtype=torch.long).random_(5)
print(target)
output = loss(input,target)
output.backward()

tensor([[ 0.1914,  0.2295, -0.7246, -0.8332,  0.8764],
        [-0.6386,  0.3057, -0.5610, -0.5233, -0.0610],
        [-0.4628,  1.3686,  2.0509, -0.6301, -1.1665]], requires_grad=True)
tensor([1, 1, 3])


### 优化器
动量算法中每一步下降都是由前面的下降方向的一个累积和当前点的梯度的方向组合而成

NAG算法进行了改进，先按照历史梯度走一步，然后按照当前点梯度走

方法有AdaGrad，RMSProp，Adam

In [12]:
torch.cuda.device_count()

0